In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nltk
import math

data = pd.read_csv("dataAfterPOS.csv")
stemmedData = pd.read_csv("stemmed.csv")
data['description'] = data['description'].str.replace('[^\w\s]','')
stemmedData['description'] = stemmedData['description'].str.replace('[^\w\s]','')


# Pre processing (always good for everything you do): MUST do

In [ ]:
#work on data NOT stemmed
from nltk.corpus import stopwords
stop = stopwords.words('english')

data["stopwords"] = data["description"].apply(lambda x: len([x for x in x.split() if x in stop]))

#remove most frequent and less frequent bcs useless
freq = pd.Series(' '.join(data['description']).split()).value_counts()[:10]
freq = list(freq.index)
data['description'] = data['description'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

unfreq = pd.Series(' '.join(data["description"]).split()).value_counts()
count = 0
for el in unfreq: 
    if(el < 10):
        count = count + 1
        
unfreqList = unfreq[-count:]
unfreq = dict(unfreqList)  
data["description"] = data['description'].apply(lambda x: " ".join(x for x in x.split() if x not in unfreq))

data.drop(["word_count","stopwords"],axis=1, inplace=True)


In [2]:
#work on STEMMED data
from nltk.corpus import stopwords
stop = stopwords.words('english')

stemmedData["stopwords"] = stemmedData["description"].apply(lambda x: len([x for x in x.split() if x in stop]))

#remove most frequent and less frequent bcs useless
freq = pd.Series(' '.join(stemmedData['description']).split()).value_counts()[:10]
freq = list(freq.index)
stemmedData['description'] = stemmedData['description'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

unfreq = pd.Series(' '.join(stemmedData["description"]).split()).value_counts()
count = 0
for el in unfreq: 
    if(el < 10):
        count = count + 1
        
unfreqList = unfreq[-count:]
unfreq = dict(unfreqList)  
stemmedData["description"] = stemmedData['description'].apply(lambda x: " ".join(x for x in x.split() if x not in unfreq))

stemmedData.drop(["word_count","stopwords"],axis=1, inplace=True)


# Tf + idf + tf-idf on whole vanilla dataset (no stemming = "basic" words) !!!need to run one of the first two cells

In [ ]:
fullText = pd.Series(' '.join(data['description']).split()).value_counts()
#getValueCounts
tfData = {}
for idx, row in data.iterrows(): 
    tfData[idx] = pd.Series(row['description'].split()).value_counts();

In [ ]:
#compute tf for every word
tf1 = {}
for idx in tfData.keys():
    tf1[idx] = {}
    for word in tfData[idx].keys():
        tf1[idx][word] = tfData[idx][word] / fullText[word]

In [ ]:
desc = []
for idx, row in data.iterrows():
    modded = []
    temp = row["description"].split()
    for word in temp:
        modded.append(tf1[idx][word])
    desc.append(modded)
#desc[0]

In [ ]:
data["description_values_tf"] = desc 

In [ ]:
#idf
docCount = dict.fromkeys(fullText.keys(),0)
idfDict1 = {}
N = len(data)                                #numero di documenti #dizionario contenente tutte le parole presenti 
idfData = dict(tfData)
for el in idfData:     #per ogni descrizione
    for word,val in idfData[el].items():
        docCount[word] = docCount[word] + 1
        
for word in fullText.keys():
    idfDict1[word] = math.log10(N/docCount[word])  

In [ ]:
#compute tf-idf
tfIdf1 = {}
for idx in tfData.keys():
    tfIdf1[idx] = {}
    for word in tfData[idx].keys():
        val = tf1[idx][word] * idfDict1[word]
        tfIdf1[idx][word] = val
#tfIdf1[0]

In [ ]:
desc1 = []
for idx, row in data.iterrows():
    modded = []
    temp = row["description"].split()
    for word in temp:
        modded.append(tfIdf1[idx][word])
    desc1.append(modded)
#desc1[0]

In [ ]:
data["description_values_tfIdf"] = desc1

In [ ]:
data.to_csv("data_tf_tfIdf_fullDataset.csv", encoding='utf-8', index=False)

# tf + idf + tf_idf on whole dataset (stemmed) !!! need to run one of the first two cells

In [ ]:
fullText = pd.Series(' '.join(stemmedData['description']).split()).value_counts()
#getValueCounts
tfData = {}
for idx, row in stemmedData.iterrows(): 
    tfData[idx] = pd.Series(row['description'].split()).value_counts();

In [ ]:
#compute tf for every word
tf1 = {}
for idx in tfData.keys():
    tf1[idx] = {}
    for word in tfData[idx].keys():
        tf1[idx][word] = tfData[idx][word] / fullText[word]

In [ ]:
desc = []
for idx, row in stemmedData.iterrows():
    modded = []
    temp = row["description"].split()
    for word in temp:
        modded.append(tf1[idx][word])
    desc.append(modded)
#desc[0]

In [ ]:
stemmedData["description_values_tf"] = desc 

In [ ]:
#idf
docCount = dict.fromkeys(fullText.keys(),0)
idfDict1 = {}
N = len(stemmedData)                                #numero di documenti #dizionario contenente tutte le parole presenti 
idfData = dict(tfData)
for el in idfData:     #per ogni descrizione
    for word,val in idfData[el].items():
        docCount[word] = docCount[word] + 1
        
for word in fullText.keys():
    idfDict1[word] = math.log10(N/docCount[word])  

In [ ]:
#compute tf-idf
tfIdf1 = {}
for idx in tfData.keys():
    tfIdf1[idx] = {}
    for word in tfData[idx].keys():
        val = tf1[idx][word] * idfDict1[word]
        tfIdf1[idx][word] = val
#tfIdf1[0]

In [ ]:
desc1 = []
for idx, row in stemmedData.iterrows():
    modded = []
    temp = row["description"].split()
    for word in temp:
        modded.append(tfIdf1[idx][word])
    desc1.append(modded)
#desc1[0]

In [ ]:
stemmedData["description_values_tfIdf"] = desc1

In [ ]:
stemmedData.to_csv("data_tf_tfIdf_stemmedDataset.csv", encoding='utf-8', index=False)

# tf + tfidf on stemmed dataset grouped by variety (run all below)

In [3]:
data = pd.read_csv("stemmed.csv")
#remove punctuation
data['description'] = data['description'].str.replace('[^\w\s]','')
data["word_count"] = data["description"].apply(lambda x: len(str(x).split(" ")))
data = data[data["word_count"] >= 3] 

#remove most frequent words, not influent bcs very frequent
freq = pd.Series(' '.join(data['description']).split()).value_counts()[:10]

freq = list(freq.index)
data['description'] = data['description'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

from nltk.corpus import stopwords
stop = stopwords.words('english')

data["stopwords"] = data["description"].apply(lambda x: len([x for x in x.split() if x in stop]))

unfreq = pd.Series(' '.join(data["description"]).split()).value_counts()
count = 0
for el in unfreq: 
    if(el < 10):
        count = count + 1
print(count)   #elementi poco frequenti sotto il 10 sono spesso typos/parole unite o robe brutte

unfreqList = unfreq[-count:]
#unfreqList

unfreq = dict(unfreqList)  #con dictionary is veeeeeeeeeeeery fasteroni
data["description"] = data['description'].apply(lambda x: " ".join(x for x in x.split() if x not in unfreq))

26449


In [4]:
fullText = pd.Series(' '.join(data['description']).split()).value_counts()

totalWords = pd.Series("".join(data["description"]).split()).value_counts().sum()
print(totalWords)

t1 = data.groupby(["variety"])
t2 = pd.DataFrame({'text' : t1['description'].apply(lambda x: (x + " ").sum())}).reset_index()
t2["word_count"] = t2["text"].apply(lambda x: len(str(x).split(" ")))
t2 = t2.sort_values('word_count',ascending = False)
#t2.iloc[0]["text"]
#t2

1752815


In [5]:
varietyDict = {}
for idx, row in t2.iterrows():
    varietyDict[row["variety"]] = pd.Series(row['text'].split()).value_counts()

In [6]:
tfDict = {}
for var in varietyDict:
    temp = {}
    for word in varietyDict[var].keys():
        temp[word] = varietyDict[var][word] / fullText[word]   #compute tf= numero di occorrenze della parola nel documento /numero di occorrenze della stessa parola in tutti i documenti 
    tfDict[var] = temp
#tfDict["Pinot Gris"] 

In [7]:
varList = data["variety"].unique().tolist()

def computeIdf1(dataset):
    a = []
    idf = {}
    for idx,row in dataset.iterrows():
        el = row["variety"]
        docList = data[data["variety"] == el]
        N = len(docList)
        idfDict = {}
        temp = t2[t2["variety"] == el]
        idfDict = dict.fromkeys(temp.iloc[0]["text"].split(),0)
        
        for idx,row in docList.iterrows():
            for word in row["description"].split():
                idfDict[word] += 1
        
        for word, val in idfDict.items():
            idfDict[word] = math.log10(N/float(val))
        a.append([el,N,idfDict])    
        idf[el] = idfDict
    return idf, a


In [8]:
idf,a = computeIdf1(t2)

In [9]:
tfIdf = {}
for var in varietyDict:
    temp = {}
    for word in tfDict[var].keys():
        temp[word] = tfDict[var][word] * idf[var][word]
    tfIdf[var] = temp 

In [10]:
#example
dictlist = []
for word,value in tfIdf["Chardonnay"].items():
    temp = [word,value]
    dictlist.append(temp)
dictlist.sort(key=lambda x: x[1],reverse=True)
dictlist
#con lo stemming cambia un po' le cose

[['puligni', 3.030032704936171],
 ['chablisienn', 3.030032704936171],
 ['chablisian', 3.030032704936171],
 ['fourchaum', 2.9160893526293346],
 ['chablislik', 2.8259127222802465],
 ['chablisstyl', 2.7995837835578974],
 ['serein', 2.751279103983342],
 ['pulignymontrachet', 2.7395866847964507],
 ['cortoncharlemagn', 2.723862885127889],
 ['kumeu', 2.723862885127889],
 ['fuissé', 2.707813410202252],
 ['clarifi', 2.5288492475044158],
 ['cob', 2.525027254113476],
 ['chassagnemontrachet', 2.3307943884124396],
 ['meursault', 2.2930398924095043],
 ['pansear', 2.1530763598325686],
 ['movi', 2.1293932122670474],
 ['challah', 2.084628478629485],
 ['brie', 2.055209252567452],
 ['leflaiv', 2.0385296042003502],
 ['mâcon', 2.0214834883497703],
 ['butteredtoast', 2.0200218032907804],
 ['durel', 1.9672343059256976],
 ['butterscotchi', 1.933825895150436],
 ['brewer', 1.9242724418118327],
 ['brulé', 1.9242724418118327],
 ['went', 1.9194829596136305],
 ['marcona', 1.9092668334379292],
 ['daffodil', 1.895458

In [11]:
desc1 = []
for idx, row in data.iterrows():
    modded = []
    temp = row["description"].split()
    for word in temp:
        modded.append(tfIdf[row["variety"]][word])
    desc1.append(modded)


In [12]:
data["description_tfIdf"] = desc1
data.drop(["region_2","word_count","stopwords"],inplace=True,axis=1)

In [13]:
data.to_csv("data_varietytfIdf_stemmedData.csv", encoding='utf-8', index=False)